In [11]:
import os
import cv2 as cv
import argparse
from sklearn.svm import LinearSVC
from skimage import feature
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from utils import read_img
from sklearn.metrics import f1_score
import joblib
from scipy.spatial.distance import cdist
from sklearn.preprocessing import StandardScaler
import helper
from sklearn.metrics import f1_score

In [12]:
train_image_paths = []
train_labels = []
# get all the image folder paths
Categories=['black widow', 'captain america', 'doctor strange', 'hulk', 'ironman', 'loki', 'spider-man', 'thanos']
datadir='dataset\\train' 

for i in Categories:
    
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        train_image_paths.append(os.path.join(path,img))
        train_labels.append(Categories.index(i))
    print(f'loaded category:{i} successfully')

loading... category : black widow
loaded category:black widow successfully
loading... category : captain america
loaded category:captain america successfully
loading... category : doctor strange
loaded category:doctor strange successfully
loading... category : hulk
loaded category:hulk successfully
loading... category : ironman
loaded category:ironman successfully
loading... category : loki
loaded category:loki successfully
loading... category : spider-man
loaded category:spider-man successfully
loading... category : thanos
loaded category:thanos successfully


In [13]:
test_image_paths = []
test_labels = []
# get all the image folder paths
Categories=['black widow', 'captain america', 'doctor strange', 'hulk', 'ironman', 'loki', 'spider-man', 'thanos']
datadir='dataset\\test' 

for i in Categories:
    
    print(f'loading... category : {i}')
    path=os.path.join(datadir,i)
    for img in os.listdir(path):
        test_image_paths.append(os.path.join(path,img))
        test_labels.append(Categories.index(i))
    print(f'loaded category:{i} successfully')

loading... category : black widow
loaded category:black widow successfully
loading... category : captain america
loaded category:captain america successfully
loading... category : doctor strange
loaded category:doctor strange successfully
loading... category : hulk
loaded category:hulk successfully
loading... category : ironman
loaded category:ironman successfully
loading... category : loki
loaded category:loki successfully
loading... category : spider-man
loaded category:spider-man successfully
loading... category : thanos
loaded category:thanos successfully


In [14]:
scaler = StandardScaler()
def bag_of_words_SIFT(image_paths, codebook):
    orb = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        keypoints, descriptors = orb.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [15]:
codebook_SIFT = joblib.load('Saved_Models_SVM/codebook_SIFT.joblib')
svm_SIFT = joblib.load('Saved_Models_SVM/svm_bow_SIFT.joblib')

C:\Users\lukas\anaconda3\envs\py10\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
print('Generating BOW features for training set...')
train_images_SIFT = bag_of_words_SIFT(train_image_paths, codebook_SIFT)
train_images_scaled_SIFT = scaler.fit_transform(train_images_SIFT)
print('Train images:', train_images_SIFT.shape)

Generating BOW features for training set...
Train images: (2325, 150)


# Gaussian pixel noise

In [17]:
def bag_of_words_SIFT_GPN(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.gaussian_pixel_noise(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [18]:
l = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [19]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_GPN(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
2
4
6
8
10
12
14
16
18


In [20]:
res

[0.27820592360365454,
 0.2291332211371926,
 0.2593962479506895,
 0.24300920009790547,
 0.2294307582301845,
 0.25059529039786743,
 0.2466749748624241,
 0.24261286242003285,
 0.24777720095098915,
 0.24976133488638919]

In [21]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.278206
2,0.229133
4,0.259396
6,0.243009
8,0.229431
10,0.250595
12,0.246675
14,0.242613
16,0.247777
18,0.249761


In [22]:
df.to_csv("Robustness_Testing_Results_SVM/Gaussian pixel noise.csv")

# Gaussian blurring

In [23]:
def bag_of_words_SIFT_GB(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.gaussian_blur(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [24]:
res = []
for i in range(10):
    print(i)
    test_images = bag_of_words_SIFT_GB(test_image_paths, codebook_SIFT, i)
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
1
2
3
4
5
6
7
8
9


In [25]:
d = {'f-1': res}
df = pd.DataFrame(data=d)
df

,f-1
0,0.278206
1,0.256647
2,0.236458
3,0.221048
4,0.205456
5,0.222273
6,0.190207
7,0.177914
8,0.181289
9,0.153725


In [26]:
df.to_csv("Robustness_Testing_Results_SVM/Gaussian blurring.csv")

# Image Contrast Increase 

In [27]:
def bag_of_words_SIFT_ICI(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.scale_contrast(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [28]:
l = [1.0, 1.01, 1.02, 1.03, 1.04, 1.05, 1.1, 1.15, 1.20, 1.25]

In [29]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_ICI(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

1.0
1.01
1.02
1.03
1.04
1.05
1.1
1.15
1.2
1.25


In [30]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
1.00,0.278206
1.01,0.271130
1.02,0.274672
1.03,0.265610
1.04,0.281646
1.05,0.252820
1.10,0.255731
1.15,0.269882
1.20,0.267684
1.25,0.260673


In [31]:
df.to_csv("Robustness_Testing_Results_SVM/Image Contrast Increase.csv")

# Image Contrast Decrease

In [32]:
l = [ 1.0, 0.95, 0.90, 0.85, 0.80, 0.60, 0.40, 0.30, 0.20, 0.10]

In [33]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_ICI(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

1.0
0.95
0.9
0.85
0.8
0.6
0.4
0.3
0.2
0.1


In [34]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
1.00,0.278206
0.95,0.271954
0.90,0.263375
0.85,0.266366
0.80,0.263801
0.60,0.272300
0.40,0.250884
0.30,0.257211
0.20,0.245193
0.10,0.268139


In [35]:
df.to_csv("Robustness_Testing_Results_SVM/Image Contrast Decrease.csv")

# Image Brightness Increase

In [36]:
def bag_of_words_SIFT_IB(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.change_brightness(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [37]:
l = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [38]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_IB(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
5
10
15
20
25
30
35
40
45


In [39]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.278206
5,0.255938
10,0.262173
15,0.260312
20,0.232420
25,0.220818
30,0.219503
35,0.222061
40,0.213327
45,0.191334


In [40]:
df.to_csv("Robustness_Testing_Results_SVM/Image Brightness Increase.csv")

# Image Brightness Decrease

In [41]:
l = [0, -5, -10, -15, -20, -25, -30, -35, -40, -45]

In [42]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_IB(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
-5
-10
-15
-20
-25
-30
-35
-40
-45


In [43]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.278206
-5,0.266581
-10,0.267993
-15,0.278687
-20,0.264330
-25,0.267444
-30,0.265710
-35,0.267769
-40,0.244018
-45,0.243000


In [44]:
df.to_csv("Robustness_Testing_Results_SVM/Image Brightness Decrease.csv")

# Occlusion of the Image Increase

In [45]:
def bag_of_words_SIFT_OII(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.occlusion(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [46]:
l = [0, 5, 10, 15, 20, 25, 30, 35, 40, 45]

In [47]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_OII(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0
5
10
15
20
25
30
35
40
45


In [48]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0,0.278206
5,0.261746
10,0.262876
15,0.273177
20,0.265432
25,0.259253
30,0.262649
35,0.263044
40,0.265577
45,0.253402


In [49]:
df.to_csv("Robustness_Testing_Results_SVM/Occlusion of the Image Increase.csv")

# Salt and Pepper Noise 

In [50]:
def bag_of_words_SIFT_SPN(image_paths, codebook, i):
    sift = cv.SIFT_create(nfeatures=300)
    codebook_size = codebook.shape[0]
    image_features = []
    for image_path in image_paths:
        img = read_img(image_path, mono=True)
        img = helper.salt_and_pepper(img, i)
        img = cv.normalize(img, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
        keypoints, descriptors = sift.detectAndCompute(img, None)
        bow = np.zeros(codebook_size)
        if descriptors is not None:
            distances = cdist(descriptors, codebook)
            for d in distances:
                bow[np.argmin(d)] += 1
        image_features.append(bow.reshape(1, codebook_size))
    image_features = np.concatenate(image_features)
    return image_features

In [51]:
l =[0.00, 0.02, 0.04, 0.06, 0.08, 0.10, 0.12, 0.14, 0.16, 0.18]

In [52]:
res = []
for i in range(len(l)):
    print(l[i])
    test_images = bag_of_words_SIFT_SPN(test_image_paths, codebook_SIFT, l[i])
    test_images_scaled = scaler.transform(test_images)
    test_predictions = svm_SIFT.predict(test_images_scaled)
    res.append(f1_score(test_labels, test_predictions, average='macro'))

0.0
0.02
0.04
0.06
0.08
0.1
0.12
0.14
0.16
0.18


In [53]:
d = {'f-1': res}
df = pd.DataFrame(data=d, index = l)
df

,f-1
0.00,0.278206
0.02,0.244745
0.04,0.239451
0.06,0.222626
0.08,0.176814
0.10,0.174893
0.12,0.155597
0.14,0.156653
0.16,0.128439
0.18,0.144291


In [54]:
df.to_csv("Robustness_Testing_Results_SVM/Salt and Pepper Noise.csv")